Importa bibliotecas, **frameworks** e arquivos .py<br>
keras: rede neural<br>
numpy: estrutura de dados<br>
pandas: estrutura de dados<br>
sklearn: normalização de dados


In [1]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from keras.models import load_model

np.random.seed(7)

%run Python/simulation.py
%run Python/dataNumpy.py
import sys

from sklearn.preprocessing import Normalizer
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
headerInput = ['Hora', 'Temperature', 'Wind_Speed', 'Wind_Direction', 'Humidity', 'Air_Density', 'Rain_Status', 'Jardim_PMV','Jardim_Temp','Sala_PMV','Sala_Temp','Mezanino_PMV','Mezanino_Temp']
headerOutput = ['PortaSalaNorte','JanelaSalaNorte','PortaSalaLeste','JanelaSalaSul','JanelaSalaOeste','PortaCozinhaLeste','JanelaCozinhaSul','JanelaMezaninoNorte','JanelaMezaninoSul','JanelaMezaninoOeste']

print(len(headerInput))
print(len(headerOutput))

13
10


Carrega **dataset**<br>
**Normaliza** os dados<br>
Divide em **trainingset e testset**

In [3]:
Input = pd.read_csv("Selected/v01_input.csv")
Output = pd.read_csv("Selected/v01_output.csv")

Input_pd = Input[headerInput]
Output_pd = Output[headerOutput]

scaler_in = Normalizer().fit(Input_pd)
Input_Norm = scaler_in.transform(Input_pd)

#scaler_out = Normalizer().fit(Output_pd)
#Output_Norm = scaler_out.transform(Output_pd)

Output_Norm = np.array(Output_pd)

print("Data Set: " + str(Input_Norm.shape[0]))
print("Training Set [70%]: "+ str(int(Input_Norm.shape[0]*0.7)+1) )
print("Test Set[30%]: " + str(int(Output_Norm.shape[0]*0.3))+ "\n")


X_training = Input_Norm[:-int(Input_Norm.shape[0]*0.3)]
Y_training = Output_Norm[:-int(Output_Norm.shape[0]*0.3)]

X_test = Input_Norm[int((Input_Norm.shape[0]*0.7)+1):]
Y_test = Output_Norm[int( (Output_Norm.shape[0]*0.7)+1) :]

print "X_training.shape: " + str(X_training.shape)
print "Y_training.shape: " + str(Y_training.shape)
print "X_test.shape: " + str(X_test.shape)
print "Y_test.shape: " + str(Y_test.shape)

Data Set: 37986
Training Set [70%]: 26591
Test Set[30%]: 11395

X_training.shape: (26591, 13)
Y_training.shape: (26591, 10)
X_test.shape: (11395, 13)
Y_test.shape: (11395, 10)


Função que permite alterar a PORTA da conexão TCP entre o energy plus, matlab e bcvtb

In [4]:
def changePort(PORT):
    base = open('simulateAndExit_base.m', 'r')
    f = open('simulateAndExit.m', 'w')
    lines = base.readlines()
    lines[37] = str(lines[37]).replace('50001', str(PORT))
    with open('simulateAndExit.m', 'w') as file_handler:
        for item in lines:
            file_handler.write("{}".format(item))
            
    base.close()
    f.close()

Modelo de rede neural testada

In [30]:
model = Sequential()
model.add(Dense(13, input_shape=(13, ), kernel_initializer='random_normal', activation='linear'))
model.add(Dense(27, activation='softsign'))
model.add(Dense(40, activation='softsign'))
model.add(Dense(10, activation='sigmoid'))
model.summary()

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.3)

# For a mean squared error regression problem
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['mae', 'acc'])

model.fit(X_training, Y_training, validation_data=(X_test, Y_test), epochs=35, batch_size=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 13)                182       
_________________________________________________________________
dense_36 (Dense)             (None, 27)                378       
_________________________________________________________________
dense_37 (Dense)             (None, 40)                1120      
_________________________________________________________________
dense_38 (Dense)             (None, 10)                410       
Total params: 2,090
Trainable params: 2,090
Non-trainable params: 0
_________________________________________________________________
Train on 26591 samples, validate on 11395 samples
Epoch 1/35
26591/26591 [==============================] - 0s - loss: 9.9894 - mean_absolute_error: 0.2855 - acc: 0.0896 - val_loss: 9.8970 - val_mean_absolute_error: 0.2731 - val_acc: 0.0914
Epoch 2/35
26591/26591 [=======

Regristra a rede neural, altera a porta da conexão TCP 

In [31]:
ID = "023"
PORT = int("50"+ID)
changePort(PORT)

model.save("Output_RN/v01_RN_"+ ID +"_model.h5")
# serialize model to JSON
model_json = model.to_json()
with open("Output_RN/v01_RN_"+ ID +"_jason.json", "w") as json_file:
    json_file.write(model_json)
    
json_file.close()

In [32]:
## SIMULACAO
Simulate = Connection(PORT)
connec = Simulate.start()

NP_data = np.zeros((1, 23))
NP_saida = np.zeros((1, 10))

while True:
    data = connec[0].recv(1024)
    #print ("raw data:")
    #print (data)
    
    if not data:
        break

    #dados direto do matlab em string
    fromMatlab = data.decode("utf-8")

    #lista com
    dataAndTime = fromMatlab.split(",")

    weatherInText = dataAndTime[0].replace("[", "")
    weatherInText = weatherInText.replace("]", "")

    timeInfo = myTime.getTimeInfo(int(dataAndTime[1]))


    #Gera uma lista com os dados climatologicos
    weather = []
    for x in weatherInText.split(" "):
        weather.append(float(x))

    ##        Hora, TempExterna,
    """
    'Hora',
    'TempExterna', 'ArVeloc', 'ArDirecao', 'Umidade', 'ArDensidade', 'StatusChuva',
    'PMV_JardimInverno', 'Temp_JardimInverno',
    'PMV_SalaCozinha', 'Temp_SalaCozinha',
    'PMV_Mezanino', 'Temp_Mezanino'
    """
    
    """
    print "timeInfo:"
    print timeInfo
    
    print "weather: " + str(len(weather))
    print weather[19]
    """
    
    inputRN = pd.DataFrame([[timeInfo[2], weather[0], weather[1], weather[2], weather[3], weather[7], weather[8],
                weather[19], weather[20], weather[23], weather[24], weather[25], weather[26]]], columns=headerInput)
    
    inputRN_np = np.array(inputRN)
    
    #print "PMV SALA: " + str(weather[23])
    
    np_inputRN = scaler_in.transform(inputRN_np)
    
    #print "np_inputRN.shape: " + str(np_inputRN.shape)
    #print "np_inputRN:"
    #print np_inputRN
    
    
    #np_outputRN = model.predict_on_batch(np_inputRN)
    np_outputRN = model.predict(np_inputRN, verbose=0)

    #print "np_outputRN:"
    #print np_outputRN
    
    np_timeInfo = np.array([timeInfo[2]])
    np_weather = np.array([weather])

    sendList = np_outputRN.tolist()[0]

    np_save = np.column_stack((inputRN_np, np_outputRN))
    

    
    NP_data = np.row_stack((NP_data, np_save))
    NP_saida = np.row_stack((NP_saida, np_outputRN))
    
    #l = pd.Series(np_save[0,:], index = Input + Output)

    
    sendToMatlab = ', '.join(str(float("%.2f" % e)) for e in sendList )

    #print("OUTPUT DATA:")
    #print (sendToMatlab)
    connec[0].sendall(str.encode(sendToMatlab))

   


Simulate.close()

arquivo = pd.DataFrame(NP_data)
arquivo.columns=headerInput+headerOutput
arquivo = arquivo[2:]
arquivo = arquivo[:-1]
arquivo.to_csv("Output_RN/v01_RN_"+ ID +".csv", index=False)


saida = pd.DataFrame(NP_saida)
saida.columns=headerOutput
saida = saida[2:]
saida = saida[:-1]
saida.to_csv("Output_RN/v01_RN_"+ ID +"_saida.csv", index=False)



print ("FIM!")


PYTHON || simulacao.py: Iniciando simulacao
PYTHON || simulacao.py: Esperando inicializacao do BCVTB
PYTHON || simulation.py: Recebendo dados
[[ 0.          0.0622225   0.00632771  0.94915682  0.27068546  0.00414989
   0.          0.          0.08429889  0.          0.08519949  0.
   0.08681331]]
[[ 0.          0.19683237  0.02280375  0.          0.88814607  0.01424329
   0.         -0.01011439  0.24700246 -0.01439612  0.23941907 -0.00803913
   0.23027116]]
[[ 0.0116637   0.18778563  0.02566015  0.          0.8864415   0.01386879
   0.         -0.01852655  0.25082679 -0.02633514  0.2453811  -0.0222861
   0.23089345]]
[[ 0.02266867  0.17794909  0.02720241  0.          0.89541261  0.0134933
   0.         -0.01654824  0.24669365 -0.02338027  0.24322497 -0.02098703
   0.20942158]]
[[ 0.03307745  0.16869501  0.02866713  0.          0.90411704  0.01314203
   0.         -0.01586669  0.23781459 -0.02223687  0.23333319 -0.02418278
   0.19923489]]
[[ 0.04343668  0.16180163  0.03149159  0.       

[[ 0.          0.11210181  0.00361619  0.84377703  0.47613132  0.00706883
   0.         -0.00716457  0.12920451 -0.01092915  0.12497782 -0.00715609
   0.12717774]]
[[ 0.00598429  0.10711888  0.00359058  0.83780126  0.49071217  0.00704362
   0.         -0.00795297  0.12537653 -0.0116661   0.12089591 -0.00816417
   0.12295021]]
[[ 0.01187764  0.10274161  0.00356329  0.83143503  0.50479984  0.00700477
   0.         -0.00871372  0.12194485 -0.01253436  0.11722151 -0.00915138
   0.119189  ]]
[[ 0.01772738  0.09868243  0.00413639  0.82727784  0.51409408  0.00698592
   0.         -0.00941668  0.11981381 -0.01333781  0.1136236  -0.01003516
   0.11647881]]
[[ 0.02337133  0.09406962  0.00408998  0.81799669  0.53169785  0.00692813
   0.         -0.00989105  0.11591493 -0.01413812  0.1105921  -0.01071734
   0.11242466]]
[[ 0.02896667  0.08979666  0.00405533  0.81106663  0.54457331  0.00688441
   0.         -0.01056009  0.11283425 -0.01466669  0.10741076 -0.01149034
   0.10916589]]
[[ 0.03554027  0

[[  1.84613912e-01   1.80600566e-01   3.45147748e-02   7.22402263e-01
    5.45815043e-01   9.29376322e-03   0.00000000e+00  -2.26879765e-05
    1.96568216e-01  -4.68563167e-03   1.90168516e-01  -4.19877504e-04
    1.93440289e-01]]
[[ 0.          0.17823276  0.03050831  0.72256524  0.57805219  0.00926992
   0.         -0.00151106  0.1955702  -0.00625082  0.18888944 -0.00222908
   0.19234939]]
[[ 0.00788885  0.17355471  0.02682209  0.70999653  0.59955263  0.00911648
   0.         -0.001893    0.19059381 -0.006539    0.18426061 -0.00271175
   0.18741369]]
[[ 0.01541716  0.16727615  0.02235488  0.69377203  0.62439483  0.00890872
   0.         -0.00219216  0.18557092 -0.00668368  0.17930096 -0.00306343
   0.18267209]]
[[ 0.02270682  0.16197535  0.01816546  0.68120473  0.64336002  0.00875055
   0.         -0.00243012  0.18067641 -0.00677999  0.17467309 -0.0032711
   0.17796801]]
[[ 0.02988249  0.15688305  0.01942362  0.67235592  0.65741468  0.00865552
   0.         -0.00272925  0.1758484  -0

[[  6.03630640e-02   9.44504412e-02   7.10153694e-03   9.58707486e-01
    2.02393803e-01   4.02895992e-03   0.00000000e+00   2.76648936e-03
    9.55881347e-02   3.94167563e-04   9.23683671e-02   4.36206353e-03
    9.79265616e-02]]
[[  6.37897250e-02   9.17863265e-02   6.73335986e-03   9.56845875e-01
    2.12632417e-01   4.02972830e-03   0.00000000e+00   2.48424096e-03
    9.50472672e-02   3.11234328e-04   9.21767743e-02   3.55311309e-03
    9.55434886e-02]]
[[  6.71839463e-02   8.94607285e-02   6.01119520e-03   9.54719237e-01
    2.22767822e-01   4.02775632e-03   0.00000000e+00   2.22831537e-03
    9.44922368e-02   2.29831878e-04   9.19246526e-02   2.79497261e-03
    9.44704148e-02]]
[[  6.03379443e-02   7.42156714e-02   4.52534582e-03   9.65407108e-01
    1.99115216e-01   3.44419957e-03   0.00000000e+00   1.72668178e-03
    8.04221398e-02   1.40103576e-04   7.84591907e-02   2.01815275e-03
    8.08123298e-02]]
[[  6.32003950e-02   7.25299771e-02   5.11622245e-03   9.63053638e-01
    2.

[[  8.97900911e-02   1.54438957e-01   1.31692134e-02   8.38040850e-01
    4.36978443e-01   6.78660612e-03   0.00000000e+00   4.24374579e-03
    1.59715350e-01  -5.42002296e-05   1.55622913e-01   4.25485624e-03
    1.57609121e-01]]
[[  9.49188628e-02   1.46530994e-01   1.36445865e-02   8.30540049e-01
    4.56797027e-01   6.75966867e-03   0.00000000e+00   4.32454660e-03
    1.56318608e-01   1.47911958e-04   1.51348563e-01   3.73498753e-03
    1.53141579e-01]]
[[  9.99981309e-02   1.38232710e-01   1.35291589e-02   8.23514019e-01
    4.76461683e-01   6.73310147e-03   0.00000000e+00   3.50060365e-03
    1.50742640e-01  -6.60020305e-04   1.45008210e-01   2.73567050e-03
    1.47315125e-01]]
[[ 0.10493024  0.12999691  0.0139907   0.8161241   0.49550392  0.00670369
   0.          0.00217669  0.14512277 -0.00199614  0.13882542  0.00134241
   0.14115041]]
[[  1.09671356e-01   1.22370144e-01   1.44304415e-02   8.08104725e-01
    5.13723718e-01   6.67274021e-03   0.00000000e+00   8.02609810e-04
   

...

[[  5.62273245e-02   6.66695419e-02   2.40974248e-03   9.63896991e-01
    2.19554315e-01   3.02106133e-03   0.00000000e+00   1.59315768e-03
    7.11768839e-02   2.21110781e-04   6.95793741e-02   2.08602855e-03
    7.18529301e-02]]
[[  5.88304109e-02   6.57830958e-02   2.13928767e-03   9.62679451e-01
    2.24625205e-01   3.01978428e-03   0.00000000e+00   1.52238689e-03
    7.11317198e-02   2.43287333e-04   6.95616584e-02   1.83579148e-03
    7.13117867e-02]]
[[  6.14267075e-02   6.48986518e-02   1.86950849e-03   9.61461508e-01
    2.29682471e-01   3.02167977e-03   0.00000000e+00   1.52393795e-03
    7.09088665e-02   3.14319787e-04   6.93860739e-02   1.62341437e-03
    7.06574750e-02]]
[[  0.00000000e+00   6.41163548e-02   1.60290887e-03   9.61745322e-01
    2.37764816e-01   3.02427867e-03   0.00000000e+00   1.49379291e-03
    6.98453646e-02   3.22638761e-04   6.83664614e-02   1.44636915e-03
    6.95300724e-02]]
[[  2.67015727e-03   6.35497429e-02   1.06806291e-03   9.61256616e-01
    2.

[[ 0.13763289  0.16456107  0.00418883  0.83776543  0.41289867  0.00672462
   0.          0.00639119  0.16338646  0.00386151  0.1586783   0.0098942
   0.17190188]]
[[ 0.          0.15844265  0.00358738  0.83705549  0.44244362  0.00674631
   0.          0.00556118  0.16088914  0.00316201  0.15548865  0.00861953
   0.16805369]]
[[ 0.00592399  0.15106186  0.0023696   0.82935922  0.46207156  0.00670781
   0.          0.00476475  0.15928618  0.00221804  0.15396585  0.00747136
   0.16305447]]
[[ 0.01170158  0.1433443   0.00175524  0.81911031  0.4856154   0.00665359
   0.          0.00452171  0.15728177  0.00207127  0.152097    0.00610001
   0.15675176]]
[[ 0.01732748  0.13630952  0.00115517  0.8086158   0.50827279  0.00658749
   0.          0.00448467  0.15377333  0.00217377  0.1485094   0.00484944
   0.15206698]]
[[  2.27549407e-02   1.28565415e-01   5.68873517e-04   7.96422923e-01
    5.34741106e-01   6.51585307e-03   0.00000000e+00   3.99840690e-03
    1.46789155e-01   1.70850589e-03   1.4

[[ 0.          0.13837258  0.01744192  0.81395638  0.50000178  0.00663526
   0.          0.00504358  0.15398803  0.00210663  0.14778026  0.00394377
   0.14960369]]
[[ 0.00578013  0.13178697  0.01676238  0.80921823  0.51443159  0.00662337
   0.          0.00422068  0.14813767  0.0010645   0.1422624   0.00298869
   0.1434876 ]]
[[  1.14830320e-02   1.25739201e-01   1.66503964e-02   8.03812241e-01
    5.28219473e-01   6.60276896e-03   0.00000000e+00   2.79888939e-03
    1.43469095e-01  -3.66182721e-04   1.37297741e-01   1.42343640e-03
    1.38430437e-01]]
[[  1.71573062e-02   1.20101144e-01   1.65853960e-02   8.00674290e-01
    5.37595595e-01   6.60253336e-03   0.00000000e+00   1.61937006e-03
    1.39153649e-01  -1.66947637e-03   1.32683385e-01   1.00874452e-04
    1.33791972e-01]]
[[  2.27100222e-02   1.14117862e-01   1.58970156e-02   7.94850778e-01
    5.50718039e-01   6.57838342e-03   0.00000000e+00   4.01454866e-04
    1.34571595e-01  -3.03023738e-03   1.27837423e-01  -1.25407968e-03


[[  6.22004317e-02   6.63471271e-02   6.22004317e-03   9.47816102e-01
    2.75459055e-01   3.34801980e-03   0.00000000e+00   2.06800486e-03
    7.77341138e-02   4.98696703e-04   7.50938376e-02   1.84546183e-03
    7.58722971e-02]]
[[  6.52091200e-02   6.63947404e-02   5.92810182e-03   9.48496291e-01
    2.72692684e-01   3.35136944e-03   0.00000000e+00   1.92524343e-03
    7.73097857e-02   3.75923588e-04   7.45726374e-02   1.40678161e-03
    7.57820118e-02]]
[[  6.81120394e-02   6.60390643e-02   5.92278604e-03   9.47645766e-01
    2.75409551e-01   3.35257738e-03   0.00000000e+00   1.70391136e-03
    7.68330278e-02   1.42247913e-04   7.43292780e-02   1.22476572e-03
    7.50851223e-02]]
[[  0.00000000e+00   6.78290683e-02   5.35492644e-03   9.51986923e-01
    2.67746322e-01   3.40448775e-03   0.00000000e+00   1.57335766e-03
    7.72743232e-02   4.25184020e-05   7.50559952e-02   1.02132604e-03
    7.60494078e-02]]
[[  2.98144424e-03   6.91695063e-02   4.77031078e-03   9.54062156e-01
    2.

[[ 0.          0.10354165  0.02119273  0.84770938  0.47229523  0.00713645
   0.         -0.0042042   0.1310721  -0.00924785  0.12135679 -0.0068837
   0.12266682]]
[[ 0.00600094  0.09541497  0.01800282  0.84013181  0.49207721  0.00710489
   0.         -0.00549099  0.12466725 -0.01073291  0.1151013  -0.00825694
   0.11627637]]
[[ 0.01188489  0.08735391  0.01426186  0.831942    0.51105008  0.00707525
   0.         -0.00706322  0.11921281 -0.01235182  0.10871652 -0.00989146
   0.1111019 ]]
[[ 0.01758985  0.07856801  0.01114024  0.82085984  0.53355889  0.00701549
   0.         -0.00837991  0.11276324 -0.01395055  0.10281263 -0.01118625
   0.10569968]]
[[ 0.02318552  0.07071583  0.00753529  0.81149318  0.55065608  0.00697514
   0.         -0.00984568  0.10785373 -0.01537754  0.09801795 -0.01244434
   0.10433004]]
[[ 0.02856238  0.06283725  0.00456998  0.79974677  0.57124769  0.00690583
   0.         -0.01098554  0.10179003 -0.01649301  0.09187782 -0.01281364
   0.09944166]]
[[ 0.03490364  0.